In [ ]:
import geopandas as gpd
import pandas as pd
import os
import matplotlib.pyplot as plt
import subprocess
import json
import pysal.viz.mapclassify as mc
import numpy as np
import re

In [ ]:
place="singapore"

In [ ]:
# Build file path
file_path = f'./out/{place}/final/weighted_difference_between_clusters.json'

# Load JSON file
with open(file_path, 'r') as file:
    colours = json.load(file)

In [ ]:
buildings = gpd.read_parquet(f"./out/{place}/buildings.pq")
tessellation = gpd.read_parquet(f"./out/{place}/tessellation_stats_clusters.pq")
streets = gpd.read_parquet(f"./out/{place}/streets.pq")

In [ ]:
tessellation.head()

In [ ]:
tessellation_colours = tessellation
tessellation_colours['colour_values'] = tessellation['cluster_ID'].astype(str).str.split('.').str[0].map(colours)

In [ ]:
tessellation = gpd.read_parquet(f"./out/{place}/tessellation_stats_no_percentile.pq")

df = tessellation.drop(['uID', 'geometry'], axis=1)

t = gpd.read_parquet(f"./out/{place}/tessellation_stats_clusters.pq")

tessellation = tessellation.merge(t.drop(["geometry"], axis=1), on='uID')

In [ ]:
# Map for comparison
f, ax = plt.subplots(figsize=(200, 200))
tessellation_colours.plot(column='cluster_ID', linewidth=0.8, ax=ax, edgecolor='0.8')
streets.plot(ax=ax, color='blue')
buildings.plot(ax=ax, color='black', alpha=0.5)
ax.set_axis_off()

# Show the plot
plt.show()

In [ ]:
def deal_with_limit_string (value):
    limits = [float(i) for i in value[1:-1].split(',')]

    if value[0] == "(":
        limits[0] = limits[0] + 0.01

    if value[-1] == ")":
        limits[1] = limits[1] - 0.01
    
    return limits

# Create a new DataFrame to store the results
jenks_df = {f"{place}":{}}

# Iterate through the columns of the GeoDataFrame
for column in tessellation.columns:
    jenks_df[place][column] = {}
    # Skip non-numeric columns
    if not tessellation[column].dtype in ['int', 'float']:
        continue

    values = tessellation[column].dropna()
    
    # Exclude infinite values
    values = values.replace([np.inf, -np.inf], np.nan).dropna()
    
    classifier = mc.NaturalBreaks(values, k=8)  # k is the number of classes

    jenks_df[place][column]["classes"] = [deal_with_limit_string(bucket) for bucket in classifier.get_legend_classes()]
    jenks_df[place][column]["min"] = str(values.min())
    jenks_df[place][column]["max"] = str(values.max())

with open(f'./out/{place}/final/jenks.json', "w") as outfile:
    json.dump(jenks_df, outfile)

In [ ]:
file = f'./out/{place}/final/tessellation.geojson'

if os.path.exists(file):
    os.remove(file)
    print("File deleted successfully.")
else:
    print("File not found.")

In [ ]:
def set_geojson_precision_and_trim_write(n_coords: int, n_trim: int, file_path: str) -> tuple:

    with open(file_path, 'r') as f:
        data = json.load(f)
    
    def set_precision_and_trim(coords, n):
        if isinstance(coords, list):
            return [set_precision_and_trim(c, n) for c in coords]
        elif isinstance(coords, dict):
            return {k: set_precision_and_trim(v, n) for k, v in coords.items()}
        elif isinstance(coords, float):
            return round(coords, n)
        else:
            if isinstance(coords, str) and coords.isdigit():
                return coords  # Don't trim strings that are only digits
            try:
                return float(coords)  # Leave other numbers untouched
            except (ValueError, TypeError):
                return coords
    
    for i,feature in enumerate(data['features']):
        for key in data['features'][i]["properties"].keys():
            if type(data['features'][i]["properties"][key]) == float:
                data['features'][i]["properties"][key] = round(data['features'][i]["properties"][key], n_trim)
            else:
                continue
            
        try:
            data['features'][i]["geometry"]["coordinates"] = set_precision_and_trim(data['features'][i]["geometry"]["coordinates"], n_coords)
        except:
            print("test")

    # Write the modified data to a new file with the name appended with "_trimmed"
    path, ext = os.path.splitext(file_path)
    new_file_path = path + '_trimmed' + ext
    with open(new_file_path, 'w') as f:
        json.dump(data, f)
    
    return os.path.basename(new_file_path), new_file_path

In [ ]:
# Define the name and filepath of the GeoJSON file to upload

tessellation.to_crs("epsg:4326").to_file(file, driver='GeoJSON')

file_name, file_path = set_geojson_precision_and_trim_write(7, 3, file)

In [ ]:
file = f'./out/{place}/final/buildings.geojson'

if os.path.exists(file):
    os.remove(file)
    print("File deleted successfully.")
else:
    print("File not found.")

In [ ]:
buildings.set_crs(tessellation.crs).to_crs("epsg:4326").to_file(file, driver='GeoJSON')

In [ ]:
file_name, file_path = set_geojson_precision_and_trim_write(7, 3, file)

In [ ]:
tessellation.drop(["geometry"], axis=1).to_csv(f'./out/{place}/final/{place}-tess.csv', index=False)

In [ ]:
import csv

def shorten_to_three_decimals(value):
    if value:
        return round(float(value), 3)
    else:
        return ""

def shorten_csv_values(input_file, output_file):
    with open(input_file, 'r') as file:
        reader = csv.reader(file)
        rows = list(reader)

    updated_rows = []
    updated_rows.append(rows[0])  # Append the first row as it is (header)

    for row in rows[1:]:
        updated_row = [str(shorten_to_three_decimals(value)) for value in row]
        updated_rows.append(updated_row)

    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(updated_rows)

    print(f"Values in '{input_file}' (excluding the first row) have been shortened to three decimal places (ignoring empty cells) and saved to '{output_file}'.")

# Example usage
input_file = f'./out/{place}/final/{place}-tess.csv'  # Replace with your input CSV file path
output_file = f'./out/{place}/final/{place}-tess.csv'  # Replace with the desired output CSV file path

shorten_csv_values(input_file, output_file)